"I/we certify that the code and data in this assignment were generated
independently, using only the tools and resources defined in the course
and that I/we did not receive any external help, coaching or contributions
during the production of this work."

References: 
1. https://github.com/lvwerra/trl/blob/master/nbs/04-gpt2-sentiment-ppo-training.ipynb
2. https://github.com/behavioral-data/Empathy-Mental-Health

In [3]:
!pip install trl tensorflow-text tf-models-official wandb==0.12.9

     |████████████████████████████████| 1.7 MB 7.3 MB/s 
     |████████████████████████████████| 97 kB 6.0 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=7e865a8d8d99f8dc44044ff33f83c5663e7846c17be35b4e39559f5c7ca606da
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
Successfully built subprocess32
  Attempting uninstall: wandb
    Found existing installation: wandb 0.12.16
    Uninstalling wandb-0.12.16:
      Successfully uninstalled wandb-0.12.16


In [1]:
!wandb login

wandb: Currently logged in as: bpn (use `wandb login --relogin` to force relogin)


In [2]:
import wandb
import torch
from itertools import chain
from torch.utils.data import DataLoader
from torch.nn import functional as F
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import copy as deepcopy
import os
import pandas as pd
from nltk.cluster import cosine_distance
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [3]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, get_polynomial_decay_schedule_with_warmup

In [4]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
path_to_repo = '/content/drive/MyDrive/UB-RL'
os.chdir(path_to_repo+'/EDG_DeepRL')

# co_path = "/content/drive/MyDrive/UB-RL/cls_roberta-large_supervised_shuffle.bin"
er_path = "/content/drive/MyDrive/UB-RL/new_ER_model.pth"
ip_path = "/content/drive/MyDrive/UB-RL/new_IP_model.pth"
ex_path = "/content/drive/MyDrive/UB-RL/new_EX_model.pth"

In [6]:
from src.model import *
from src.rewards import *

ER path: /content/drive/MyDrive/UB-RL/new_ER_model.pth
IP path: /content/drive/MyDrive/UB-RL/new_IP_model.pth
EX path: /content/drive/MyDrive/UB-RL/new_EX_model.pth
No GPU available, using the CPU instead.
Loading Empathy Model on device: cpu
Empathy Model Loaded


In [7]:
tokenizer_dict = load_tokenizer()

Loading the tokenizer...


In [8]:
tokenizer = tokenizer_dict['tok']

In [9]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8' 

map_name_to_handle = {
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
}

map_model_to_preprocess = {
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

In [67]:
def build_encoder():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=False, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']

  return tf.keras.Model(text_input, net)

In [80]:
encoder = build_encoder()

In [86]:
def calc_reward(seeker_post, generated_response, tokenizer, original_response = None, teacher_forcing = False):
    '''
    TO BE IMPLEMENTED-

    Given sentence and response calculate the reward.
    '''
    emp_score = calc_empathy_score(seeker_post, generated_response)
    flow_score = informationFlow([generated_response])
    if(teacher_forcing):
      orig_res_enc = encoder(tf.constant(original_response)).numpy().squeeze()
      gen_res_enc = encoder(tf.constant(generated_response)).numpy().squeeze()
      jacc_score = edit_level_jaccard(original_response, generated_response)/2
      cos_dist = (1-cosine_distance(orig_res_enc, gen_res_enc))
      print(f"jacc_dist: {jacc_score}, cos_dist: {cos_dist}")

      return 0.5*emp_score + 0.225*flow_score + 0.225*cos_dist + 0.05*jacc_score
    # coh_score = calc_coherence_score(original_responses, generated_responses)
    return 0.625*emp_score + 0.375*flow_score

In [87]:
calc_reward(["Hi How are you"], ["I'm doing good. Just hanging out."], tokenizer=tokenizer, original_response = ["I'm doing good. how are you?"], teacher_forcing=True)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


jacc_dist: 0.19047619047619047, cos_dist: 0.9261377575128665


0.2179048049642045

In [21]:
ckpt_path = "/content/drive/MyDrive/UB-RL/gpt2/best_ckpt_epoch=4_valid_loss=2.5959.ckpt"
model = load_model(ckpt_path, tokenizer_dict['vocab_size'])

Loading Model...


In [22]:
# model.eval()

In [101]:
class Conversation:

    def __init__(self, seq_gen, tokenizer_dict):
        self.seq_gen = seq_gen
        self.history = list()
        self.tokenizer_dict = tokenizer_dict
        self.input_history = list()

    def step(self, sentence):
        self.history.append("<sp1> "+sentence)
        # input = preprocess(self.history)
        response, _ = next_sentence_eval(self.seq_gen, sentence, self.tokenizer_dict, input_history=self.input_history)
        reward = calc_reward([sentence], [response], self.tokenizer_dict['tok'])
        self.history.append("<sp2> "+response)
        return response
    
    def train(self, seq_gen, episodes = 100, lr=0.001, batch_size = 32, seed = 10):
        pass 

    def render(self):
        for i, sentence in enumerate(self.history):
            if(self.history[i].split()[0]=="<sp1>"):
                print(f"You: {' '.join(sentence.split()[1:])}")
            else:
                print(f"Bot: {' '.join(sentence.split()[1:])}")
    
    def reset(self):
        self.history.clear()
        self.input_history.clear()
        return self.history

In [94]:
new_conv = Conversation(model, tokenizer_dict)

In [95]:
new_conv.reset()

[]

In [ ]:
new_conv.step('I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people, we felt like the only people in the world.')

"That's amazing! I'm glad you didn't go too far away from your friend and had fun!"

In [ ]:
new_conv.step('This was a best friend. I miss her.')

"Oh that's good to hear. It is a great idea to always keep in touch and get to know each other in a more meaningful way!"

In [ ]:
new_conv.step("We no longer talk.")

'Well that sounds like a nice change.'

In [ ]:
new_conv.render()

You: I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people, we felt like the only people in the world.
Bot: That's amazing! I'm glad you didn't go too far away from your friend and had fun!
You: This was a best friend. I miss her.
Bot: Oh that's good to hear. It is a great idea to always keep in touch and get to know each other in a more meaningful way!
You: We no longer talk.
Bot: Well that sounds like a nice change.


## Training with PPO

In [102]:
from transformers import GPT2Tokenizer
from trl.gpt2 import GPT2HeadWithValueModel, respond_to_batch
from trl.ppo import PPOTrainer

vocab_size = tokenizer_dict['vocab_size']
gpt2_model = GPT2HeadWithValueModel.from_pretrained('gpt2')
gpt2_model.resize_token_embeddings(vocab_size)
gpt2_model_ref = GPT2HeadWithValueModel.from_pretrained('gpt2')
gpt2_model_ref.resize_token_embeddings(vocab_size)
# gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

Some weights of GPT2HeadWithValueModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'v_head.summary.weight', 'lm_head.weight', 'v_head.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2HeadWithValueModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias',

Embedding(50260, 768)

In [104]:
gpt2_model.load_state_dict(model.state_dict(), strict=False)
gpt2_model_ref.load_state_dict(model.state_dict(), strict=False)

_IncompatibleKeys(missing_keys=['v_head.summary.weight', 'v_head.summary.bias'], unexpected_keys=[])

In [106]:
new_df=pd.read_csv('../processed_dataset.csv')

In [107]:
new_df.drop(["Unnamed: 0"], axis = 1, inplace = True)
new_df.head()

,query,response
0,I remember going to see the fireworks with my ...,"Was this a friend you were in love with, or ju..."
1,This was a best friend. I miss her.,Where has she gone?
2,We no longer talk.,Oh was this something that happened because of...
3,it feels like hitting to blank wall when i se...,Oh ya? I don't really see how
4,dont you feel so.. its a wonder,I do actually hit blank walls a lot of times b...


In [108]:
from tqdm.auto import tqdm
tqdm.pandas()

## Supplementary

In [47]:
def add_generated_response(dataset, tokenizer):
    # get model response
  responses = list()

  for query_txt in tqdm(dataset['query'].to_list()):
    response, _ = next_sentence_eval(model, query_txt, tokenizer_dict, [], device = 'cuda')
    responses.append(response)

  dataset['generated_response'] = responses
  return dataset

In [49]:
new_df = add_generated_response(new_df[:10000], tokenizer)

  0%|          | 0/10000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [50]:
new_df.to_csv('../processed_df_10000.csv', index = False)

## Important

In [109]:
new_df

,query,response
0,I remember going to see the fireworks with my ...,"Was this a friend you were in love with, or ju..."
1,This was a best friend. I miss her.,Where has she gone?
2,We no longer talk.,Oh was this something that happened because of...
3,it feels like hitting to blank wall when i se...,Oh ya? I don't really see how
4,dont you feel so.. its a wonder,I do actually hit blank walls a lot of times b...
...,...,...
29662,"Thank you, I am trying my best.",If you need to you can always take it again. G...
29663,What a scary night that was.,What happened? Hope you are okay?
29664,"My house burned night earlier, had to get my m...","Omg that is so scary, I am so sorry to hear th..."
29665,I woke up this morning to my wife telling me s...,Oh hey that's awesome! That is awesome right?


In [116]:
ppo_config = {"steps": 1000,
    "batch_size": 8,
    "forward_batch_size": 8,
    "ppo_epochs": 4,   
    "init_kl_coef":0.2,
    "target": 2,
    "horizon":10000,
    "gamma":0.95,
    "vf_coef":.1}

In [117]:
wandb.init(name='run2', project='gpt2-empathetic_dialogue', config= ppo_config)

wandb: wandb version 0.12.16 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [118]:
## Offline learning
dataset = new_df # processed dataset

def train(df, ppo_config):
  ppo_trainer = PPOTrainer(gpt2_model, gpt2_model_ref, **ppo_config)
  # fbs = ppo_config['forward_batch_size']

  for epoch in tqdm(range(int(np.ceil(ppo_config["steps"]/ppo_config['batch_size'])))):
    torch.cuda.empty_cache()
    logs = dict()
    game_data = dict()
    timing = dict()

    df_batch = df.sample(ppo_config['batch_size'])
    game_data['query'] = df_batch['query'].to_list()
    df_batch['query'] = df_batch['query'].apply(lambda x: "<sp1> "+x)
    # print(df_batch['query'].to_list())
    df_tokens = tokenizer(df_batch['query'].to_list(), padding=True, return_tensors='pt')['input_ids']
    # print(df_tokens)
    
    # print(df_batch['tokens'])
    query_tensors = torch.from_numpy(np.array(df_tokens))
    # print(f"query: {query_tensors}")

    # response_tensors = []
    # for i in range(int(ppo_config['batch_size'])):
    response = respond_to_batch(gpt2_model, query_tensors, top_p=0.92)
    # response_tensors.append(response)
    # response_tensors = torch.cat(response_tensors)
    # print(f"response_tensor: {response}")
    game_data['response'] = [tokenizer.decode(response[i], skip_special_tokens=True) for i in range(ppo_config['batch_size'])]
    # print(game_data['response'])

    rewards = []
    for i in range(int(ppo_config['batch_size'])):
      # print(f"query: {game_data['query'][i]}, generated_response: {game_data['response'][i]}, original_response: {df_batch['response'].iloc[i]}")
      res = calc_reward([game_data['query'][i]], [game_data['response'][i]], tokenizer, [df_batch['response'].iloc[i]])
      # print(res)
      rewards.append(res)
    rewards = torch.from_numpy(np.array(rewards))
    # print(f"rewards: {rewards}")

    stats = ppo_trainer.step(query_tensors, response, rewards)

    table_rows = [list(r) for r in zip(game_data['query'], game_data['response'], rewards.cpu().tolist())]
    logs.update({'game_log':wandb.Table(
        columns=['query', 'response', 'reward'],
        rows=table_rows)})
    logs.update(timing)
    logs.update(stats)
    logs['env/reward_mean'] = torch.mean(rewards).cpu().numpy()
    logs['env/reward_std'] = torch.std(rewards).cpu().numpy()
    logs['env/reward_dist'] = rewards.cpu().numpy()
    wandb.log(logs)

In [ ]:
train(new_df[:200], ppo_config)

  0%|          | 0/125 [00:00<?, ?it/s]

## Evaluation

In [49]:
new_upg_conv = Conversation(gpt2_model, tokenizer_dict)

In [50]:
new_upg_conv.step("I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people, we felt like the only people in the world.")

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


'Was there a lot of disagreement?'

In [51]:
new_upg_conv.step("This was a best friend. I miss her.")

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


'I bet you were glad to see her.'

In [52]:
new_upg_conv.step("We no longer talk.")

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


'Maybe we should go and get more drinks.'

In [54]:
new_upg_conv.render()

You: I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people, we felt like the only people in the world.
Bot: Was there a lot of disagreement?
You: This was a best friend. I miss her.
Bot: I bet you were glad to see her.
You: We no longer talk.
Bot: Maybe we should go and get more drinks.


In [55]:
torch.save(gpt2_model.state_dict(), 'RL_model.pth')